In [191]:
import fitz  # Import PyMuPDF
import os  # Import os for directory operations
import pickle
import pandas as pd

In [192]:
FOLDER_PATH = 'Doc/'  # Adjust this path to your folder
PICKLE_PATH = 'Saved/textdata.pkl'  # Specify the filename and path to save the data
CSV_PATH    = 'Saved/textdata.csv'

BLOCK_MIN_SENTENCES = 3
BLOCK_MAX_SENTENCES = 5
BLOCK_MIN_WORDS     = 20
SENTENCE_MIN_WORDS  = 4

In [193]:
def extract_paragraphs_from_page(page):
    blocks = page.get_text("blocks")
    paragraphs = [block[4].strip() for block in blocks if block[4].strip() != '']

    merged_paragraphs = []
    temp_paragraph = ""
    temp_word_count = 0
    temp_sentence_count = 0

    for paragraph in paragraphs:
        sentences = [sentence.strip() for sentence in paragraph.split('.') if sentence.strip()]
        for sentence in sentences:
            words = sentence.split()
            if len(words) >= SENTENCE_MIN_WORDS:
                # Count as a valid sentence
                if temp_sentence_count < BLOCK_MAX_SENTENCES:
                    temp_paragraph += (sentence + ". ")
                    temp_sentence_count += 1
                    temp_word_count += len(words)
                else:
                    # Once the sentence limit is reached, append the paragraph and reset
                    merged_paragraphs.append(temp_paragraph.strip())
                    temp_paragraph = sentence + ". "
                    temp_sentence_count = 1
                    temp_word_count = len(words)
            else:
                # If it's not a valid sentence, just append the words to the temp paragraph
                temp_paragraph += (sentence + " ")
                temp_word_count += len(words)
        
        # Handle case where paragraph ends without reaching the sentence limit
        if temp_word_count < BLOCK_MIN_WORDS:
            # Merge with the next paragraph if current is too short
            continue
        elif temp_sentence_count < BLOCK_MIN_SENTENCES:
            # Check if we can append this to previous paragraphs
            if merged_paragraphs:
                merged_paragraphs[-1] += " " + temp_paragraph.strip()
                temp_paragraph = ""
                temp_word_count = 0
                temp_sentence_count = 0
            else:
                merged_paragraphs.append(temp_paragraph.strip())
                temp_paragraph = ""
                temp_word_count = 0
                temp_sentence_count = 0
        else:
            merged_paragraphs.append(temp_paragraph.strip())
            temp_paragraph = ""
            temp_word_count = 0
            temp_sentence_count = 0

    # Make sure the last paragraph gets added if it's not empty
    if temp_paragraph.strip():
        if temp_sentence_count >= BLOCK_MIN_SENTENCES and temp_word_count >= BLOCK_MIN_WORDS:
            merged_paragraphs.append(temp_paragraph.strip())
        elif merged_paragraphs:
            # Merge with the last paragraph if conditions are not met
            merged_paragraphs[-1] += " " + temp_paragraph.strip()

    return merged_paragraphs



def extract_and_clean_text_from_folder(folder_path):
    all_pdf_data = []
    list_pdf_filenames(folder_path)
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(folder_path, filename)
            doc = fitz.open(pdf_path)
            
            for page_num in range(len(doc)):
                page = doc.load_page(page_num)
                paragraphs = extract_paragraphs_from_page(page)
                para_num = 0
                for paragraph in paragraphs:
                    para_num += 1
                    document_name = filename
                    text = ' '.join(paragraph.split())
                    word_count = len(text.split())  # Count the words in the cleaned paragraph
                    all_pdf_data.append([document_name, page_num + 1, para_num, word_count, text.strip()])
            
            doc.close()
    return all_pdf_data


def list_pdf_filenames(folder_path):
    pdf_filenames = [filename for filename in os.listdir(folder_path) if filename.lower().endswith('.pdf')]
    idx = 0
    print(f'Documents in folder:')
    for filename in pdf_filenames:
        idx += 1
        print(f'\t {idx} : {filename}')
    return pdf_filenames

def print_specific_page(all_cleaned_data, document_name, page_number):
    for entry in all_cleaned_data:
        if entry[0] == document_name and entry[1] == page_number:
            print(f"Content of page {page_number} in document '{document_name}':\n\t{entry[2]}")
            return
    print(f"Page {page_number} in document '{document_name}' not found.")

def save_data(data, filename):
    with open(filename, 'wb') as file:
        pickle.dump(data, file)
    print(f"Data saved to {filename}")

def load_data(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)


In [194]:
all_cleaned_data        = extract_and_clean_text_from_folder(FOLDER_PATH)
df = pd.DataFrame(all_cleaned_data, columns=['Document', 'Page', 'Block', "WordCount", 'Text'])
with open(CSV_PATH, mode='w', newline='', encoding='utf-8-sig') as file:
    df.to_csv(file, index=False)

word_count_stats = df.groupby('Document')['WordCount'].agg(['min', 'max', 'mean']).reset_index()
word_count_stats = word_count_stats.rename(columns={'min': 'MinWordCount', 'max': 'MaxWordCount', 'mean': 'AvgWordCount'})


save_data(all_cleaned_data, PICKLE_PATH)
loaded_data = load_data(PICKLE_PATH)

Documents in folder:
	 1 : DDDD & DD.pdf
	 2 : Latające Taksówki - Wdrażanie Innowacji  - Projekt (Lange, 67619).pdf
	 3 : Ustawa.pdf
Data saved to Saved/textdata.pkl


In [195]:
os.system(f"start EXCEL.EXE \"{CSV_PATH}\"")
word_count_stats

,Document,MinWordCount,MaxWordCount,AvgWordCount
0,DDDD & DD.pdf,20,84,33.052632
1,Latające Taksówki - Wdrażanie Innowacji - Pro...,20,167,53.733333
2,Ustawa.pdf,21,167,45.747126
